## Compare Audio Files

In [ ]:
# the code was originally used with logbot-audio env
# data visualization -> run with venv-umineko-analysis

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as tck
import seaborn as sns
import scipy.io.wavfile
# import pyaudio
import librosa
import librosa.display
import IPython.display
print(librosa.__version__)
# palette = sns.color_palette(['#E69F00', '#56B4E9', '#009E73', '#F0E442', '#0072B2', '#D55E00', '#CC79A7', '#000000'])
# palette = sns.color_palette(["#ff4554", "#00bbdf", "#bad600", "#f02d7d", "#f8b62e", "#8b26a6","#808080"])
# sns.set_palette(palette)
palette2 = sns.color_palette(["#D81B60", "#1E88E5", "#FFC107", "#004D40"])
palette = palette2
display(palette)
# https://anoiro.com/themes/switch-joycons
sns.set_theme(context='paper', style='ticks', palette=palette, font_scale=1.0)
color_palette_2 = sns.color_palette('viridis_r', n_colors=10, desat=0.75)

## Functions

In [ ]:
# # Load wav file and Normalize 8bit PCM data between -1 and 1
# def load_and_normalize_wav_8bit(audio_path):
#     rate, data = scipy.io.wavfile.read(audio_path)

#     while len(data) > 30400: # adjust to 3.8000 sec
#         data = np.delete(data, len(data)-1)

#     # if Stereo, convert it to Mono
#     if np.ndim(data) > 1:
#         data = data[:, 0] # convert it to mono data
#     else:
#         data = data
    
#     # change unsigned int to int
#     if min(data) < 0:
#         data = data.astype(float)
#     else:
#         data = data - np.mean(data)

#     data_normalized = data / 128 # normalize 8bit PCM data between -1 and 1

#     return rate, data_normalized


# Load wav file and Normalize 8bit PCM data between -1 and 1
# def load_and_normalize_wav_16bit(audio_path):
#     rate, data = scipy.io.wavfile.read(audio_path)
    
#     while len(data) > 33600: # adjust to 4.2000 sec
#         data = np.delete(data, len(data)-1)

#     # if Stereo, convert it to Mono
#     if np.ndim(data) > 1:
#         data = data[:, 0] # convert it to mono data
#     else:
#         data = data
    
#     # print(np.min(data))
#     # print(np.max(data))
#     # change unsigned int to int
#     if min(data) < 0:
#         data = data.astype(float)
#     else:
#         data = data - np.mean(data)

#     data_normalized = data / 32767 # normalize PCM data between -1 and 1

#     return rate, data_normalized


# https://www.youtube.com/watch?v=rlypsap6Wow
# Calculate the amplitude envelope
# def amplitude_envelope(signal, frame_length, hop_length):
#     amplitude_envelope = []
    
#     # calculate amplitude envelope for each frame
#     for i in range(0, len(signal), hop_length): # if hop_length = 100 -> i = 0, 100, 200, 300, ...
#         current_frame_amplitude_envelope = max(signal[i:i+frame_length]) # if frame_length = 200 -> max(signal[0-200]), max(signal[100-300]), max(signal[200-400]), ...
#         amplitude_envelope.append(current_frame_amplitude_envelope)
    
#     return np.array(amplitude_envelope)

# # Calculate the amplitude envelope in fancy way
# def fancy_amplitude_envelope(signal, frame_length, hop_length):
#     return np.array([max(signal[i:i+frame_length]) for i in range(0, signal.size, hop_length)])

# https://www.youtube.com/watch?v=EycaSbIRx-0&t=1001s
# Calculate RMS Energy
def rms (signal, frame_length, hop_length):
    rms = []

    for i in range(0, len(signal), hop_length):
        rms_current_frame = np.sqrt(np.sum(signal[i:i+frame_length]**2) / frame_length)
        rms.append(rms_current_frame)

    return np.array(rms)


# Load WAV files

In [ ]:
dir_name = "../audio/"
# dir_name = "../audio/umineko-2024/"

audio_sec = 4.2

# hayabusa_file = f"hayabusa_nr_amp4_{audio_sec}s"
# noise_file = f"noise_amp4_{audio_sec}s"

hayabusa_file = f"_hayabusa"
noise_file = f"_noise"

hayabusa_path = os.path.join(dir_name, hayabusa_file + ".wav")
print(hayabusa_path)
noise_path = os.path.join(dir_name, noise_file + ".wav")
print(noise_path)

print(f"Loading WAV file... : {hayabusa_path}")
# rate, hayabusa = load_and_normalize_wav_8bit(audio_path=hayabusa_path)
# sampling_rate, hayabusa_data = load_and_normalize_wav_16bit(audio_path=hayabusa_path)
hayabusa_data, sampling_rate = librosa.load(hayabusa_path, sr=None) # load with original sampling rate

print(hayabusa_data[:5])
print(f"sampling rate: {sampling_rate}")
print(f"Length: {len(hayabusa_data)}")
print(f"Min: {np.min(hayabusa_data)}")
print(f"Max: {np.max(hayabusa_data)}")
print(np.max(hayabusa_data))
print(f"------------------------------------------------------------------")
print(f"Loading WAV file... : {noise_path}")
# rate, noise = load_and_normalize_wav_8bit(audio_path=noise_path)
# sampling_rate, noise_data = load_and_normalize_wav_16bit(audio_path=noise_path)
noise_data, sampling_rate = librosa.load(noise_path, sr=None) # load with original sampling rate
print(noise_data[:5])
print(f"sampling rate: {sampling_rate}")
print(f"Length: {len(noise_data)}")
print(f"Min: {np.min(noise_data)}")
print(f"Max: {np.max(noise_data)}")
print(f"------------------------------------------------------------------")
print(f"Time:")
time = np.arange(0, hayabusa_data.shape[0]/sampling_rate, 1/sampling_rate) # get time to plot
print(time[:5])
print(len(time))

## AE & RMSE

In [ ]:
# fig_save_dir = "../figure/umineko-2024/"

In [ ]:
FRAME_LENGTH = 200
HOP_LENGTH = 100

# Amplitude Envelope
# ae_hayabusa = amplitude_envelope(signal=hayabusa_data, frame_length=FRAME_LENGTH, hop_length=HOP_LENGTH)
# ae_noise = amplitude_envelope(signal=noise_data, frame_length=FRAME_LENGTH, hop_length=HOP_LENGTH)
# RMS Energy
# rms_hayabusa = rms(signal=hayabusa_data, frame_length=FRAME_LENGTH, hop_length=HOP_LENGTH)
# rms_noise = rms(signal=noise_data, frame_length=FRAME_LENGTH, hop_length=HOP_LENGTH)

_rms_hayabusa = librosa.feature.rms(y=hayabusa_data, frame_length=FRAME_LENGTH, hop_length=HOP_LENGTH, center=True)
_rms_noise = librosa.feature.rms(y=noise_data, frame_length=FRAME_LENGTH, hop_length=HOP_LENGTH, center=True)

# print(rms_hayabusa.shape)
# print(_rms_hayabusa[0, :].shape)
# print(np.sum(rms_hayabusa - _rms_hayabusa[0, 1:]))

rms_hayabusa = _rms_hayabusa[0, :]
rms_noise = _rms_noise[0, :]

# transform frame to time domain
frames = range(len(rms_hayabusa))
t = librosa.frames_to_time(frames=frames, sr=sampling_rate, hop_length=HOP_LENGTH)
# print(f"Hayabusa")
describe_hayabusa = pd.DataFrame(rms_hayabusa).describe()
# display(describe_hayabusa)

# print(f"Noise")
describe_noise = pd.DataFrame(rms_noise).describe()
# display(describe_noise)

# plot
gridspec_kw={
    # 'width_ratios': [1, 1],
    'wspace': 0.1,
    'hspace': 0.8}
fig, axes = plt.subplots(2, 1, figsize=(6, 6), gridspec_kw=gridspec_kw)
ax0, ax1 = axes.flatten()
ax_list = [ax0, ax1]
rms_list = [rms_hayabusa, rms_noise]
title_list = ["Falcon", "Noise"]
y_lim_list = [100, 100]
mean_list = [np.mean(rms_hayabusa), np.mean(rms_noise)]
median_list = [np.median(rms_hayabusa), np.median(rms_noise)]
bins_list = [30, 10]
for i, ax in enumerate(ax_list):
    ax = sns.histplot(ax=ax, x=rms_list[i], color=palette[1], bins=bins_list[i], alpha=0.75)
    # ax = plt.hist(y_list[i], bins=30, alpha=0.8)
    ax.set_ylim(0, y_lim_list[i])
    ax.set_xlim(0.0, 0.65)
    ax.set_title(title_list[i], pad=10)
    ax.set_xlabel("RMS Energy", labelpad=10)
    ax.set_ylabel("Frequency", labelpad=10)
    ax.axvline(x=mean_list[i], linewidth=2, label="Mean", color="black")
    ax.text(x=0.25, y=70, s=f"Mean RMS Energy= {mean_list[i]:.5f}", fontsize=14)
    # ax.axvline(x=median_list[i], linewidth=2, label="Median", color="black")
    # ax.text(x=0.25, y=70, s=f"Median RMS Energy= {mean_list[i]:.5f}")
    ax.grid(which='major')
plt.show()
plt.close()
# fig.savefig(f"{fig_save_dir}/00_comparison_01.png", dpi=600)
# fig.savefig(f"{fig_save_dir}/00_comparison_01.svg", dpi=600)
# ---------------------------------------------------------------------
# pv = 10 に設定してから再生しているので、
# 厳密には再生された音を録音してその音の強さを比較しないといけない？
# ここでのスペック上の音の強さと実際に鳴った音は違う？
# pv = 10 のとき、ラベラーさん的には hayabusa_42_nr_amp3.mp4 が
# もっともクリアで、良く聴こえたようだった（私も同じ感覚だった）
# また、 noise_42_amp3 はhayabusa_42_nr_amp3よりも少し音が小さく感じた
# hayabusa_42_nr_amp3 と noise_42_amp4 を pv=10で再生して
# それを比較すべし（大きなズレがなければ良いはず...）
# ---------------------------------------------------------------------

# Visualization

In [ ]:
# Spectrogram
def plot_spectrogram(ax, data, rate, ax_title):
    # fft frame size
    fft_size = 2**8
    # frame shift length
    hop_length = int(fft_size / 4)  
    # Short-time Foulier Transformation
    amplitude = np.abs(librosa.core.stft(data, n_fft=fft_size, hop_length=hop_length))
    # Amplitude to db
    log_power = librosa.core.amplitude_to_db(amplitude, ref=np.max)
    # librosa.display.specshow(log_power, sr=rate, hop_length=hop_length, x_axis='time', y_axis='hz', cmap='magma')
    img = librosa.display.specshow(log_power, sr=rate, hop_length=hop_length, x_axis='time', y_axis='hz', cmap='viridis', ax=ax)
    # plt.colorbar(format='%+2.0f dB', ax=ax)
    ax.set_title(ax_title.replace("_", " "), pad=10, fontsize=24, fontweight='bold')
    ax.set_xlabel("Time [s]", labelpad=5)
    ax.set_ylabel("Frequency [Hz]",  labelpad=10)
    ax.set_xticks(np.arange(0, 5, 1))
    ax.set_xlim(0.0, 4.20)
    ax.set_yticks(np.arange(0, 5000, 1000))
    ax.set_ylim(0.0, 4000)
    
    ax.xaxis.set_major_formatter(tck.FuncFormatter(lambda x,_: f'{x:.1f}'))
    return img

In [ ]:
# plot
GRIDSPEC_KW={
    'width_ratios': [1, 1],
    'wspace': 0.4,
    'hspace': 0.6
}
fig, axes = plt.subplots(2, 2, figsize=(8, 5), gridspec_kw=GRIDSPEC_KW)
ax0, ax1, ax2, ax3 = axes.flatten()
ax_list = [ax0, ax1, ax2, ax3]
signal_list = [hayabusa_data, hayabusa_data, noise_data, noise_data]
# ae_list = [ae_hayabusa, ae_noise]
rms_list = [rms_hayabusa, rms_hayabusa, rms_noise, rms_noise]
title_list = [f"Predator", f"Predator", f"Noise", f"Noise"]
mean_y_jitter_list = [0.90] * 4
plt.rcParams['legend.handlelength'] = 1.2
for i, ax in enumerate(ax_list):
    ax = ax_list[i]
    if i == 0 or i == 2:
        # ax = sns.lineplot(ax=ax, x=time, y=signal_list[i], linewidth=1.0, color="#333333", alpha=0.5, label="Signal")
        # ax = sns.lineplot(ax=ax, x=t, y=rms_list[i], linewidth=3, color=palette[2], alpha=0.9, label="RMS")
        ax.plot(time, signal_list[i], linewidth=1.0, color="#333333", alpha=0.5, label="Signal")
        ax.plot(t, rms_list[i], linewidth=2, color=palette[2], alpha=0.9, label="RMS")
        ax.axhline(y=np.mean(rms_list[i]), xmin=0.0, xmax=4.2, linewidth=2, linestyle="-", color=palette[1], alpha=0.9)
        ax.set_xlim(-0.15, 4.35)
        ax.set_yticks(np.arange(-1, 1.1, 1.0))
        ax.set_ylim(-1.5, 1.5)
        ax.yaxis.set_major_formatter("{:.1f}".format)
        ax.set_xlabel("Time [s]", labelpad=5)
        ax.set_ylabel("Amplitude", labelpad=10)
        ax.text(x=0.10, y=np.mean(rms_list[i])+mean_y_jitter_list[i], size=8, fontweight='bold', s=f"Mean RMS = {np.mean(rms_list[i]):.5f}", color=palette[1])
        legend = ax.legend(loc="lower right", ncol=3, fontsize=8)
        for line in legend.get_lines():
            line.set_linewidth(1.0)  # 凡例の線の太さを設定
        ax.grid(which='major', alpha=0.3)
    
    elif i == 1:
        img0 = plot_spectrogram(ax=ax, data=hayabusa_data, rate=sampling_rate, ax_title=title_list[i])
        fig.colorbar(img0, ax=ax, format="%+2.f dB")
    elif i == 3:
        img1 = plot_spectrogram(ax=ax, data=noise_data, rate=sampling_rate, ax_title=title_list[i])
        fig.colorbar(img1, ax=ax, format="%+2.f dB")
    
    ax.set_title(title_list[i], pad=5, fontsize=12, fontweight='bold')

    # if i < 2:
    #     ax.set_xlabel('')
    # cbar = fig.colorbar(img0, ax=[ax1, ax3], format="%+2.f dB")
        
    # if i == 0 or i == 1:
    #     ax.set_xlabel("", labelpad=10)

    if i == 0:
        ax.text(-0.22, 1.2, '(a)', transform=ax.transAxes, fontsize=12, fontweight='bold', va='top', ha='right')
    elif i == 2:
        ax.text(-0.22, 1.2, '(b)', transform=ax.transAxes, fontsize=12, fontweight='bold', va='top', ha='right')


# plt.tight_layout()
plt.show()
# plt.close()

save_dir = f"../output/figure-for-paper"
fig.savefig(f"{save_dir}/png/fig_s02_audio.png", dpi=350, bbox_inches='tight', pad_inches=0.2, transparent=False)
fig.savefig(f"{save_dir}/pdf/fig_s02_audio.pdf", dpi=600, bbox_inches='tight', pad_inches=0.2, transparent=False)
# fig.savefig(f"{save_dir}/figure/fig_s02_audio.svg", dpi=600, bbox_inches='tight', pad_inches=0.1, transparent=True)

In [ ]:
# Duration of 1 sample
sample_duration = 1 / sampling_rate
print(f"Duration of 1 sample is: {sample_duration: .6f} seconds")

# Duration of the audio signal in seconds
duration = sample_duration * len(hayabusa_data)
print(f"Duration of signal is: {duration:.6f} seconds")

## FFT (Fast Foulier Trasnformation)

In [ ]:
# Frequency Amplitude plot
def plot_freqency_amplitude(ax, data, rate, file_name):
    # FFT (Fast Fourier transform) : Time domain to Frequency Domain
    SAMPLING_RATE = 8000
    signal = data
    # fft_data = np.abs(np.fft.fft(data))
    F = np.fft.fft(signal)
    F_abs = np.abs(F)
    F_abs_amp = F_abs / SAMPLING_RATE * 2 # normalize
    F_abs_amp[0] = F_abs_amp[0] / 2
    F_abs_amp[0] = 0
    fft_data = F_abs_amp
    nyquist_frequency = int(SAMPLING_RATE/2)
    # Frequency
    freqList = np.fft.fftfreq(data.shape[0], d=1.0/rate)  
    # plot

    # print(freqList)
    # print(fft_data)
    sns.lineplot(x=freqList, y=fft_data, ax=ax)
    # ax.set_xlim(0, rate/2) # show only 0 ～ 4000Hz
    ax.set_xlim(400, 4000)
    ax.set_ylim(0, 0.11)
    ax.set_title(file_name.replace("_", " "), fontsize=18, pad=10)
    ax.set_xlabel("Frequency [Hz]", fontsize=16, labelpad=10)
    ax.set_ylabel("Amplitude", fontsize=16, labelpad=10)
    # ax.grid(which='major')
    

plt.rcParams.update({'font.size': 16})
GRIDSPEC_KW = {
    # 'width_ratios': [1,1],
    'height_ratios': [1,1], 
    'wspace': 0.1, 
    'hspace': 0.5
}
fig, (ax0, ax1) = plt.subplots(2, 1, figsize=(9, 10), gridspec_kw=GRIDSPEC_KW)
# Hayabusa
plot_freqency_amplitude(ax=ax0, data=hayabusa_data, rate=sampling_rate, file_name=hayabusa_file)
# Noise
plot_freqency_amplitude(ax=ax1, data=noise_data, rate=sampling_rate, file_name=noise_file)
ax0.set_xscale('log')
ax1.set_xscale('log')
ax0.grid()
ax1.grid()
plt.show()
plt.close()

## Spectrogram

In [ ]:
# plt.rcParams.update({'font.size': 16})
GRIDSPEC_KW = {
    # 'width_ratios': [1,1],
    # 'height_ratios': [1,1], 
    'wspace': 0.5, 
    'hspace': 0.1
}
fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(20, 4), gridspec_kw=GRIDSPEC_KW)
# Hayabusa
img0 = plot_spectrogram(ax=ax0, data=hayabusa_data, rate=sampling_rate, ax_title="Predator")
# Noise
img1 = plot_spectrogram(ax=ax1, data=noise_data, rate=sampling_rate, ax_title="Noise")
# fig.colorbar(img0, ax=[ax0, ax1], format="%+2.f dB") # common colorbar
# fig.colorbar(img0, ax=ax0, format="%+2.f dB")
# fig.colorbar(img1, ax=ax1, format="%+2.f dB")
# Add a common colorbar
cbar = fig.colorbar(img0, ax=[ax0, ax1], format="%+2.f dB")
# cbar.set_label('Amplitude [dB]', fontsize=16)

plt.show()
plt.close()